# libraries und optionen

In [72]:
import pandas as pd
import openpyxl
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets, interact

In [73]:
# Anzeigeoptionen ändern, um vollständige Inhalte zu zeigen
pd.set_option('display.max_colwidth', None)   # unbegrenzte Spaltenbreite
pd.set_option('display.max_rows', None)       # alle Zeilen anzeigen

# analyse df erstellen

## original dokument laden

In [74]:
wahlomat_orig = pd.read_csv("original-Wahl-O-Mat Sachsen 2024_Datensatz.csv", sep=";", encoding="utf-8")
wahlomat_orig.head(20)

,Partei: Nr.,Partei: Kurzbezeichnung,Partei: Name,These: Nr.,These: Titel,These: These,Position: Position,Position: Begründung
0,1,CDU,Christlich Demokratische Union Deutschlands,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,Als Sächsische Union halten wir am gesetzlich fixierten und von allen beteiligten Akteuren vereinbarten Kohlekompromiss mit einem Ausstieg aus der Kohleverstromung bis 2038 fest. Die sächsischen Kohlereviere sind Garanten der Versorgungssicherheit und können eine Schlüsselrolle bei den notwendigen Zukunftsinvestitionen einnehmen. Dafür braucht es Ersatzinvestitionen in bestehende Standorte und Transformationsregionen und zielgerichtete staatliche Investitionsanreize.
1,2,AfD,Alternative für Deutschland,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme zu,"Solange es keine konkurrenzfähige und sowohl grundlast- als auch regelfähige Alternativen zur Braunkohleverstromung gibt, sich diese Energiequelle rentiert und die Menschen in der Region mit dem Abbau der Braunkohle bzw. der Erschließung neuer Tagebaue einverstanden sind, solange wollen wir diese Energieform beibehalten."
2,3,DIE LINKE,DIE LINKE,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,"Schon heute ist die Kohleverstromung in Teilen nicht mehr rentabel und deutlich teurer als die Energiegewinnung aus erneuerbaren Quellen. Kohleverstromung gehört zudem zu den klimaschädlichsten Formen der Energiegewinnung. Gleichzeitig setzen wir uns dafür ein, dass die betroffenen Menschen in den Kohlerevieren eine Zukunftsperspektive sowohl für ihre Arbeitsplätze als auch für ihre Region bekommen."
3,4,GRÜNE,BÜNDNIS 90/DIE GRÜNEN,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,"Der Kohleausstieg muss und wird deutlich vor 2038 kommen. In Sachsen darf nicht mehr Kohle verbrannt werden, als das deutsche 1,5-Grad-Budget erlaubt. Wegen steigender Kohle- und CO₂-Preise wird Kohle schon deutlich vor 2038 nicht mehr wettbewerbsfähig sein. Mit dem massiven Ausbau Erneuerbarer Energien wollen wir einen frühestmöglichen Ausstieg aus der Kohle ermöglichen und sicherstellen, dass Sachsen Energieland bleibt – wirtschaftsstark, mit versorgungssicherer und bezahlbarer Energie."
4,5,SPD,Sozialdemokratische Partei Deutschlands,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,Wir stehen zum vereinbarten Kohle-Ausstieg bis spätestens 2038.
5,6,FDP,Freie Demokratische Partei,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,"Wir Freie Demokraten lehnen einen vorgezogenen Kohleausstieg ab, wollen aber auch an den bisher getroffenen Vereinbarungen festhalten. Wir sollten den Kohleausstieg auch als Chance begreifen und mit neuen Technologien und gezielten Investitionen das Energieland Sachsen nachhaltig umbauen und in seiner Bedeutung ausbauen."
6,7,FREIE WÄHLER,FREIE WÄHLER,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,Die betroffenen Regionen bereiten sich gerade auf den Strukturwandel vor. Die Entscheidung zum Kohleausstieg ist für 2038 getroffen. Ein ständiges Hin und Her erschwert die Planungssicherheit. Einen vorzeitigen Ausstieg lehnen wir aus diesem Grund aber ebenso ab.
7,8,Die PARTEI,"Partei für Arbeit, Rechtsstaat, Tierschutz, Elitenförderung und basisdemokratische Initiative",1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,Nach 2038 wird in Sachsen Braunkohle angebaut.
8,9,PIRATEN,Piratenpartei Deutschland,1,Braunkohleabbau,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,stimme nicht zu,"Die sächsischen Braunkohlegebiete sind noch immer für einen sehr hohen CO₂-Ausstoß verantwortlich. Wir fordern den schnellstmöglichen Ausstieg aus der Kohleverstromung und den sofortigen Stopp

In [75]:
wahlomat_orig["Position: Position"].unique()

array(['stimme nicht zu', 'stimme zu', 'neutral'], dtype=object)

In [76]:
wahlomat_orig.isna().sum()

Partei: Nr.                 0
Partei: Kurzbezeichnung     0
Partei: Name                0
These: Nr.                  0
These: Titel                0
These: These                0
Position: Position          0
Position: Begründung       32
dtype: int64

## df mit relevanten spalten und parteien

In [77]:
# Nur relevante Spalten auswählen
alle_parteien_these_position = wahlomat_orig[["Partei: Kurzbezeichnung", "These: Titel", "Position: Position"]].copy()
alle_parteien_these_position.rename(columns={"Partei: Kurzbezeichnung": "Partei", # Spaltennamen ändern
                     "These: Titel": "Titel",
                     "These: These": "These",
                     "Position: Position": "Position"}, inplace=True)  

# Nur relevante Parteien auswählen
parteien_auswahl = ["CDU", "AfD", "DIE LINKE", "GRÜNE", "SPD", "FDP", "BSW"]

parteien = alle_parteien_these_position[alle_parteien_these_position["Partei"].isin(parteien_auswahl)].reset_index(drop=True)
parteien.head(20)

,Partei,Titel,Position
0,CDU,Braunkohleabbau,stimme nicht zu
1,AfD,Braunkohleabbau,stimme zu
2,DIE LINKE,Braunkohleabbau,stimme nicht zu
3,GRÜNE,Braunkohleabbau,stimme nicht zu
4,SPD,Braunkohleabbau,stimme nicht zu
5,FDP,Braunkohleabbau,stimme nicht zu
6,BSW,Braunkohleabbau,stimme nicht zu
7,CDU,Wählen ab 16 Jahren,stimme nicht zu
8,AfD,Wählen ab 16 Jahren,stimme nicht zu
9,DIE LINKE,Wählen ab 16 Jahren,stimme zu


In [78]:
parteien.shape

(266, 3)

## spalten mit achsen- und wertezuordnungen hinzufügen

### zweiten df laden

In [79]:
achsenzuordnung = pd.read_csv("achsen-und-werte-zuordnung-der-thesen-und-anwtworten.csv", sep=";", encoding="utf-8")
achsenzuordnung = achsenzuordnung[["These", "Titel", "Achse", "stimme zu", "neutral", "stimme nicht zu"]] # relevante Spalten
achsenzuordnung.head(20)


,These,Titel,Achse,stimme zu,neutral,stimme nicht zu
0,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Braunkohleabbau,Y,-1,0,1
1,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Wählen ab 16 Jahren,Y,1,0,-1
2,Sachsen soll sich für einen Ausbau der Autobahn A4 zwischen Görlitz und Dresden einsetzen.,Ausbau der A4,Y,-1,0,1
3,"Sachsen soll sich dafür einsetzen, dass Deutschland weiterhin Waffen an die Ukraine liefert.",Waffen für die Ukraine,Y,1,0,-1
4,"An den Schulen des Freistaats sollen weiterhin Kopfnoten (Betragen, Fleiß, Mitarbeit, Ordnung) vergeben werden.",Kopfnoten,Y,-1,0,1
5,Mehr Krankenhäuser sollen in öffentlicher Trägerschaft sein.,Krankenhäuser in öffentlicher Trägerschaft,X,-1,0,1
6,An den sächsischen Grenzen zu den europäischen Nachbarstaaten sollen weiterhin Personenkontrollen stattfinden.,Personenkontrollen an den Grenzen,Y,-1,0,1
7,"Sachsen soll sich dafür einsetzen, dass wieder eine Vermögenssteuer erhoben wird.",Vermögenssteuer,X,-1,0,1
8,In den sächsischen Lehrplänen soll die Beschäftigung mit der SED-Diktatur stärker berücksichtigt werden.,Beschäftigung mit der SED-Diktatur,Y,1,0,-1
9,Sächsische Hochschulen sollen stärker mit privaten Unternehmen zusammenarbeiten.,Zusammenarbeit mit privaten Unternehmen,X,1,0,-1


In [80]:
achsenzuordnung.shape

(38, 6)

### merge der beiden DataFrames

In [81]:
# merge mit df_parteien auf "Titel"
main = parteien.merge(achsenzuordnung, on="Titel", how="left")
main.head(50)

,Partei,Titel,Position,These,Achse,stimme zu,neutral,stimme nicht zu
0,CDU,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
1,AfD,Braunkohleabbau,stimme zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
2,DIE LINKE,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
3,GRÜNE,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
4,SPD,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
5,FDP,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
6,BSW,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1
7,CDU,Wählen ab 16 Jahren,stimme nicht zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1
8,AfD,Wählen ab 16 Jahren,stimme nicht zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1
9,DIE LINKE,Wählen ab 16 Jahren,stimme zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1


In [82]:
# 4. Funktion zur Ermittlung des richtigen Werts
def map_position_to_value(row):
    pos = str(row["Position"]).strip().lower()
    if pos == "stimme zu":
        return row["stimme zu"]
    elif pos == "neutral":
        return row["neutral"]
    elif pos == "stimme nicht zu":
        return row["stimme nicht zu"]
    else:
        return None

# 5. Neue Spalte "Wert" erzeugen
main["Wert"] = main.apply(map_position_to_value, axis=1)

main.head(50)

,Partei,Titel,Position,These,Achse,stimme zu,neutral,stimme nicht zu,Wert
0,CDU,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
1,AfD,Braunkohleabbau,stimme zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,-1
2,DIE LINKE,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
3,GRÜNE,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
4,SPD,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
5,FDP,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
6,BSW,Braunkohleabbau,stimme nicht zu,In Sachsen soll auch nach 2038 noch Braunkohle abgebaut werden.,Y,-1,0,1,1
7,CDU,Wählen ab 16 Jahren,stimme nicht zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1,-1
8,AfD,Wählen ab 16 Jahren,stimme nicht zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1,-1
9,DIE LINKE,Wählen ab 16 Jahren,stimme zu,Bei den Wahlen zum Sächsischen Landtag sollen Jugendliche ab 16 Jahren wählen dürfen.,Y,1,0,-1,1


In [83]:
# Speichern der Daten in einer Excel-Datei zur Überprüfung
# Speicherpfad festlegen
speicher_ordner = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\sonstiges"

# Dateiname und voller Pfad
dateiname = "check.xlsx"
voller_pfad = os.path.join(speicher_ordner, dateiname)

# Speichern als Excel
main.to_excel(voller_pfad, index=False, engine='openpyxl')

print(f"Datei erfolgreich gespeichert unter: {voller_pfad}")

Datei erfolgreich gespeichert unter: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\sonstiges\check.xlsx


# finaler analyse df

## analyse df für cluster-thesen (hat nichts mit dieser ipynb zu tun!!)

In [84]:
analyse_mit_thesen = main[["Partei", "Titel", "These", "Achse", "Wert"]]

# Speicherpfad festlegen
speicher_ordner = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\thesen"
dateiname = "analyse-mit-thesen.csv"
voller_pfad = os.path.join(speicher_ordner, dateiname)

# Speichern als CSV (ohne Excel-Engine)
analyse_mit_thesen.to_csv(voller_pfad, index=False, sep=";", encoding="utf-8")

print(f"Datei erfolgreich gespeichert unter: {voller_pfad}")

Datei erfolgreich gespeichert unter: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\thesen\analyse-mit-thesen.csv


## analyse df für main schaubilder

In [85]:
analyse = main[["Partei", "Titel", "Achse", "Wert"]]

# Speicherpfad festlegen
speicher_ordner = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\main"

# Dateiname und voller Pfad
dateiname = "analyse.xlsx"
voller_pfad = os.path.join(speicher_ordner, dateiname)

# Speichern als Excel
analyse.to_excel(voller_pfad, index=False, engine='openpyxl')

print(f"Datei erfolgreich gespeichert unter: {voller_pfad}")

# Anzeige der ersten 50 Zeilen der Analyse 
analyse.head()

Datei erfolgreich gespeichert unter: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\Daten Sachsen\main\analyse.xlsx


,Partei,Titel,Achse,Wert
0,CDU,Braunkohleabbau,Y,1
1,AfD,Braunkohleabbau,Y,-1
2,DIE LINKE,Braunkohleabbau,Y,1
3,GRÜNE,Braunkohleabbau,Y,1
4,SPD,Braunkohleabbau,Y,1


In [86]:
analyse.dtypes

Partei    object
Titel     object
Achse     object
Wert       int64
dtype: object

In [87]:
analyse.shape

(266, 4)

In [88]:
analyse['Achse'].unique()

array(['Y', 'X'], dtype=object)

# analyse

## mittelwerte berechnen

In [89]:
# X- und Y-Werte separieren
x_df = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].mean().rename("X")
y_df = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].mean().rename("Y")

# Zusammenführen zu df_coords
df_coords = pd.concat([x_df, y_df], axis=1).reset_index()
df_coords.head(7)

,Partei,X,Y
0,AfD,0.066667,-0.652174
1,BSW,-0.200000,0.043478
2,CDU,0.733333,-0.391304
3,DIE LINKE,-0.733333,0.826087
4,FDP,0.733333,0.043478
5,GRÜNE,-0.466667,0.695652
6,SPD,-0.400000,0.304348


## diagramm mit mittelwerten erstellen (plotly)

In [90]:
# Farben definieren
farben = {
    "SPD": "#dc111e",
    "GRÜNE": "#1ea12d",
    "DIE LINKE": "#ff0000",
    "CDU": "#005973",
    "FDP": "#feed01",
    "AfD": "#008fd7",
    "BSW": "#af2ac3"
}

In [91]:
# 1. Plot: Parteipositionen (Mittelwerte X/Y)
x_mean = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].mean().rename("X")
y_mean = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].mean().rename("Y")
df_coords = pd.concat([x_mean, y_mean], axis=1).reset_index()

fig1 = px.scatter(
    df_coords,
    x="X",
    y="Y",
    text="Partei",
    color="Partei",
    color_discrete_map=farben,
    title="Positionierung der Parteien im Wahl-O-Mat Sachsen 2024",
    width=900,
    height=700
)

fig1.update_traces(marker=dict(size=20), textposition='top center')
fig1.update_layout(
    title=dict(text="Positionierung der Parteien im Wahl-O-Mat Sachsen 2024", x=0.5, xanchor='center', yanchor='top', y=0.9),
    xaxis=dict(title="", showticklabels=False, showline=True, zeroline=True, showgrid=True, gridcolor="lightgrey"),
    yaxis=dict(title="", showticklabels=False, showline=True, zeroline=True, showgrid=True, gridcolor="lightgrey"),
    plot_bgcolor="#e5ecf6",
    annotations=[
        dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16)),
        dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16)),
        dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16)),
        dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))
    ]
)
fig1.show()

## diagramm mit std erstellen

#### 2. Plot: Ellipsen mit Mittelwerten + Standardabweichung von 1

In [92]:
# 2. Plot: Ellipsen mit Mittelwerten + Standardabweichung von 1
x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

fig2 = go.Figure()
angles = np.linspace(0, 2 * np.pi, 100)
scale = 1  # Standardabweichung 1σ

for _, row in df_stats.iterrows():
    x0, y0 = row["X"], row["Y"]
    a, b = row["X_std"], row["Y_std"]
    partei = row["Partei"]
    farbe = farben.get(partei, "gray")

    # Punkt mit Hoveranzeige
    fig2.add_trace(go.Scatter(
        x=[x0],
        y=[y0],
        mode='markers',
        marker=dict(size=15, color=farbe),
        name=partei,
        hovertemplate=(
            f"<b>{partei}</b><br>" +
            "X: %{x:.2f}<br>" +
            "Y: %{y:.2f}<extra></extra>"
        )
    ))

    # Statischer Parteiname oberhalb des Punkts in schwarzer Farbe
    fig2.add_trace(go.Scatter(
        x=[x0],
        y=[y0 + 0.08],
        mode="text",
        text=[partei],
        textfont=dict(size=14, color="black"),
        showlegend=False,
        hoverinfo="skip"
    ))

    # Ellipse zeichnen
    if a > 0 and b > 0:
        xs = x0 + scale * a * np.cos(angles)
        ys = y0 + scale * b * np.sin(angles)
        fig2.add_trace(go.Scatter(
            x=xs, y=ys,
            fill="toself",
            mode="lines",
            line=dict(color=farbe, width=1),
            fillcolor=farbe,
            opacity=0.25,
            hoverinfo="skip",
            showlegend=False
        ))

# Layout
fig2.update_layout(
    hovermode="closest",
    title={
        'text': "Parteipositionen mit Ellipsen (1σ)",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),
    yaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),
    plot_bgcolor="#e5ecf6",
    width=900,
    height=700
)

# Achsenbeschriftungen
fig2.add_annotation(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig2.add_annotation(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig2.add_annotation(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16))
fig2.add_annotation(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))

fig2.show()

##### gleiches diagramm, aber mit gestrichelter linie

In [113]:
# Plot: Ellipsen mit Mittelwerten + Standardabweichung
x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

fig2a = go.Figure()
angles = np.linspace(0, 2 * np.pi, 100)

for _, row in df_stats.iterrows():
    x0, y0 = row["X"], row["Y"]
    a, b = row["X_std"], row["Y_std"]
    partei = row["Partei"]
    farbe = farben.get(partei, "gray")

    fig2a.add_trace(go.Scatter(x=[x0], y=[y0], mode='markers+text',
                              marker=dict(size=15, color=farbe),
                              text=[partei], textposition="top center",
                              name=partei))
    if a > 0 and b > 0:
        xs = x0 + a * np.cos(angles)
        ys = y0 + b * np.sin(angles)
        fig2a.add_trace(go.Scatter(x=xs, y=ys, mode='lines',
                                  line=dict(color=farbe, dash='dot'),
                                  name=f"{partei} (Ellipse)",
                                  showlegend=False))

fig2a.update_layout(
    title={
        'text': "Parteipositionen mit Ellipsen (1σ)",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),
    yaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),

    plot_bgcolor="#e5ecf6",
    width=900,
    height=700
)

fig2a.add_annotation(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig2a.add_annotation(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig2a.add_annotation(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16))
fig2a.add_annotation(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))

fig2a.show()

#### 3. Plot: Ellipsen mit Mittelwerten + Standardabweichung von 0.5

In [93]:
# 3. Plot: Ellipsen mit Mittelwerten + Standardabweichung von 0.5
x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

fig3 = go.Figure()
angles = np.linspace(0, 2 * np.pi, 100)
scale = 0.5  # Standardabweichung 0.5σ

for _, row in df_stats.iterrows():
    x0, y0 = row["X"], row["Y"]
    a, b = row["X_std"], row["Y_std"]
    partei = row["Partei"]
    farbe = farben.get(partei, "gray")

    # Punkt mit Hoveranzeige
    fig3.add_trace(go.Scatter(
        x=[x0],
        y=[y0],
        mode='markers',
        marker=dict(size=15, color=farbe),
        name=partei,
        hovertemplate=(
            f"<b>{partei}</b><br>" +
            "X: %{x:.2f}<br>" +
            "Y: %{y:.2f}<extra></extra>"
        )
    ))

    # Statischer Parteiname oberhalb des Punkts in schwarzer Farbe
    fig3.add_trace(go.Scatter(
        x=[x0],
        y=[y0 + 0.08],
        mode="text",
        text=[partei],
        textfont=dict(size=14, color="black"),
        showlegend=False,
        hoverinfo="skip"
    ))

    # Ellipse zeichnen
    if a > 0 and b > 0:
        xs = x0 + scale * a * np.cos(angles)
        ys = y0 + scale * b * np.sin(angles)
        fig3.add_trace(go.Scatter(
            x=xs, y=ys,
            fill="toself",
            mode="lines",
            line=dict(color=farbe, width=1),
            fillcolor=farbe,
            opacity=0.2,
            hoverinfo="skip",
            showlegend=False
        ))

# Layout
fig3.update_layout(
    hovermode="closest",
    title={
        'text': f"Parteipositionen mit Ellipsen ({scale}σ)",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),
    yaxis=dict(
    title="",
    range=[-1.2, 1.2],
    zeroline=True,
    showgrid=True,
    gridcolor="lightgrey",
    showline=True,
    tickmode='array',
    tickvals=[-1, 1],
    ticktext=["−1", "1"]
),
    plot_bgcolor="#e5ecf6",
    width=900,
    height=700
)

# Achsenbeschriftungen
fig3.add_annotation(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig3.add_annotation(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
fig3.add_annotation(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16))
fig3.add_annotation(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))



# 4. Plot (interaktiv mit Widget für Partei): Ellipsen mit Mittelwerten + Standardabweichung von 1σ

In [94]:
# 4. Plot: ellipsen mit Mittelwerten + Standardabweichung von 1σ, interaktiv mit Widget
def plot_ellipsen(parteien):
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()
    
    # Filter auf gewählte Parteien
    df_stats = df_stats[df_stats["Partei"].isin(parteien)]
    
    fig4 = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)
    scale = 1

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")

        # Add marker
        fig4.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15, color=farbe),
            name=partei,
            text=[partei],
            hovertemplate=("<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>")
        ))

        # Add Parteiname (Text)
        fig4.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        # Add Ellipse
        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig4.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.25,
                hoverinfo="skip",
                showlegend=False
            ))

    # Layout und Annotationen wie gehabt
    fig4.update_layout(
        xaxis=dict(
            title="", range=[-1.2, 1.2], zeroline=True, showgrid=True, gridcolor="lightgrey",
            showline=True, tickmode='array', tickvals=[-1, 1], ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="", range=[-1.2, 1.2], zeroline=True, showgrid=True, gridcolor="lightgrey",
            showline=True, tickmode='array', tickvals=[-1, 1], ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        width=900,
        height=700,
        margin=dict(l=60, r=60, t=60, b=60),
        legend=dict(
            orientation="v",
            x=1.05,
            y=1,
            xanchor="left",
            yanchor="top"
        ),
        hovermode="closest",
        title={'text': "Parteipositionen mit Ellipsen (1σ)", 'x': 0.5, 'xanchor': 'center'},
    )

    fig4.add_annotation(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
    fig4.add_annotation(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
    fig4.add_annotation(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16))
    fig4.add_annotation(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))
    return fig4


# Widget für die Parteiauswahl
parteien_liste = sorted(analyse["Partei"].unique())
parteien_select = widgets.SelectMultiple(
    options=parteien_liste,
    value=tuple(parteien_liste),
    description='Partei(en):',
    disabled=False
)

interact(plot_ellipsen, parteien=parteien_select)


interactive(children=(SelectMultiple(description='Partei(en):', index=(0, 1, 2, 3, 4, 5, 6), options=('AfD', '…

<function __main__.plot_ellipsen(parteien)>

# 5. Plot (interaktiv mit Widget für Partei): Ellipsen mit Mittelwerten + Standardabweichung von 0.5σ

In [95]:
# 5. Plot: ellipsen mit Mittelwerten + Standardabweichung von 0.5σ, interaktiv mit Widget
def plot_ellipsen(parteien):
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()
    
    # Filter auf gewählte Parteien
    df_stats = df_stats[df_stats["Partei"].isin(parteien)]
    
    fig5 = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)
    scale = 0.5  # Standardabweichung 0.5σ

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")

        fig5.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15, color=farbe),
            name=partei,
            text=[partei],
            hovertemplate=("<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>")
        ))

        fig5.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig5.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.25,
                hoverinfo="skip",
                showlegend=False
            ))

    # >>> Hier kommt dein gewünschter Layout-Block rein:
    fig5.update_layout(
        xaxis=dict(
            title="", range=[-1.2, 1.2], zeroline=True, showgrid=True, gridcolor="lightgrey",
            showline=True, tickmode='array', tickvals=[-1, 1], ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="", range=[-1.2, 1.2], zeroline=True, showgrid=True, gridcolor="lightgrey",
            showline=True, tickmode='array', tickvals=[-1, 1], ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        width=900,
        height=700,
        margin=dict(l=60, r=60, t=60, b=60),
        legend=dict(
            orientation="v",
            x=1.05,
            y=1,
            xanchor="left",
            yanchor="top"
        ),
        hovermode="closest",
        title={'text': "Parteipositionen mit Ellipsen (0.5σ)", 'x': 0.5, 'xanchor': 'center'},
    )
    # <<< Ende Layout

    fig5.add_annotation(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
    fig5.add_annotation(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16))
    fig5.add_annotation(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16))
    fig5.add_annotation(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16))
    return fig5

# Widget für die Parteiauswahl
parteien_liste = sorted(analyse["Partei"].unique())
parteien_select = widgets.SelectMultiple(
    options=parteien_liste,
    value=tuple(parteien_liste),
    description='Partei(en):',
    disabled=False
)

interact(plot_ellipsen, parteien=parteien_select)


interactive(children=(SelectMultiple(description='Partei(en):', index=(0, 1, 2, 3, 4, 5, 6), options=('AfD', '…

<function __main__.plot_ellipsen(parteien)>

# Diagramme speichern 1: Ellipse 0,5 std, je für alle Parteien einzeln und für alle Parteien zusammen

## 1. PDF A2 Ausdruck

In [96]:
# Funktion zum Plotten und Speichern der Ellipsen für eine bestimmte Partei
# Diese Funktion kann für jede Partei aufgerufen werden, um die Ellipsen zu generieren
def plot_partei_ellipse(partei_name, analyse, farben, save_path_base):
    scale_factor = 8  # Für A2
    scale = 0.5       # Standardabweichung

    # Statistikdaten vorbereiten
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

    # Nur die gewünschte Partei auswählen
    df_stats = df_stats[df_stats["Partei"] == partei_name]

    # Leeres Diagramm starten
    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        farbe = farben.get(partei_name, "gray")

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei_name,
            hovertemplate=(
                f"<b>{partei_name}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"
            ),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei_name],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    # Layout und Achsenbeschriftung
    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=f"{partei_name} – Position mit Ellipse ({scale}σ)",
            x=0.5,
            xanchor="center",
            font=dict(size=20 * scale_factor)
        ),
        font=dict(size=8 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=400, b=100, l=100, r=100),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )

    # Datei speichern
    filename = f"{save_path_base}\\ellipsen_a2_{partei_name.lower().replace(' ', '_')}.pdf"
    fig.write_image(filename, width=7016, height=4960)
    print(f"{partei_name}: gespeichert unter {filename}")


In [97]:
# Funktion anwenden
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck"

for partei in parteien:
    plot_partei_ellipse(partei, analyse, farben, save_path)

SPD: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_spd.pdf
CDU: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_cdu.pdf
FDP: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_fdp.pdf
GRÜNE: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_grüne.pdf
DIE LINKE: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_die_linke.pdf
AfD: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\pdf-a2-querformat-ausdruck\ellipsen_a2_afd.pdf
BSW: gespeichert unter C:\User

## 2. PNG. Webseite und Social Media, 1600 x 1100 (anpassen über width und height)

In [98]:
def plot_partei_ellipse_png(parteien_liste, analyse, farben, save_path_base, all_together=False):
    scale_factor = 2  # Für Web reicht Faktor 2-3 meist aus (bei Bedarf anpassen)
    scale = 0.5

    # Statistikdaten vorbereiten
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

    # Parteien filtern
    if not all_together:
        df_stats = df_stats[df_stats["Partei"].isin(parteien_liste)]

    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei,
            hovertemplate=(
                f"<b>{partei}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"
            ),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    # Titel für Einzelpartei oder alle zusammen
    if all_together:
        title_str = "Alle Parteien – Positionen mit Ellipsen (0.5σ)"
        filename = f"{save_path_base}\\ellipsen_webseite_alle_parteien.png"
    else:
        title_str = f"{parteien_liste[0]} – Position mit Ellipse (0.5σ)"
        filename = f"{save_path_base}\\ellipsen_webseite_{parteien_liste[0].lower().replace(' ', '_')}.png"

    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=title_str,
            x=0.5,
            xanchor="center",
            font=dict(size=20 * scale_factor)
        ),
        font=dict(size=10 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=120, b=60, l=60, r=60),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )

    # Größe für Web (z.B. 1600x1100px)
    fig.write_image(filename, width=1600, height=1100)
    print(f"PNG gespeichert: {filename}")

In [99]:
# Anwendung für einzelne Parteien:
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path_png = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100"

for partei in parteien:
    plot_partei_ellipse_png([partei], analyse, farben, save_path_png, all_together=False)

# Anwendung für alle zusammen:
plot_partei_ellipse_png(parteien, analyse, farben, save_path_png, all_together=True)

PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100\ellipsen_webseite_spd.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100\ellipsen_webseite_cdu.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100\ellipsen_webseite_fdp.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100\ellipsen_webseite_grüne.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webseite-1600x1100\ellipsen_webseite_die_linke.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\png-social media und webse

## 3. SVG (vektorbasiert, skalierbar), 800x600

ggf. anpassen:
für WEB: 1600x1100 (für header oä)
für DRUCK: z. B. 3508×2480 für A4, 7016×4960 für A2 bei 300 dpi

Vorsicht: ggf. scale_factor bearbeiten

In [100]:
def plot_partei_ellipse_svg(parteien_liste, analyse, farben, save_path_base, all_together=False,
                            width=1600, height=1100, scale_factor=2,
                            title_str_override=None, filename_override=None):
    scale = 0.5
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()
    if not all_together:
        df_stats = df_stats[df_stats["Partei"].isin(parteien_liste)]

    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")
        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei,
            hovertemplate=(f"<b>{partei}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"),
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))
        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    if all_together:
        title_str = title_str_override or "Alle Parteien – Positionen mit Ellipsen (0.5σ)"
        filename = filename_override or f"{save_path_base}\\ellipsen_webseite_alle_parteien.svg"
    else:
        title_str = title_str_override or f"{parteien_liste[0]} – Position mit Ellipse (0.5σ)"
        filename = filename_override or f"{save_path_base}\\ellipsen_webseite_{parteien_liste[0].lower().replace(' ', '_')}.svg"

    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=title_str,
            x=0.5,
            xanchor="center",
            font=dict(size=18 * scale_factor)
        ),
        font=dict(size=8 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=120, b=60, l=60, r=60),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )
    fig.write_image(filename, width=width, height=height)
    print(f"SVG gespeichert: {filename}")


In [101]:
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path_svg = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600"

# Einzelne Parteien (Standardgröße für Web, beliebig anpassen)
for partei in parteien:
    plot_partei_ellipse_svg([partei], analyse, farben, save_path_svg, width=800, height=600, scale_factor=1)

# Alle zusammen
plot_partei_ellipse_svg(parteien, analyse, farben, save_path_svg, all_together=True, width=800, height=600, scale_factor=1)


SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_spd.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_cdu.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_fdp.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_grüne.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_die_linke.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\svg-vektor-800x600\ellipsen_webseite_afd.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachs

# Diagramme speichern 2: Ellipse 1 std, je für alle Parteien einzeln und für alle Parteien zusammen

## 1. PDF A2 Ausdruck

In [102]:
# Funktion zum Plotten und Speichern der Ellipsen für eine bestimmte Partei
# Diese Funktion kann für jede Partei aufgerufen werden, um die Ellipsen zu generieren
def plot_partei_ellipse(partei_name, analyse, farben, save_path_base):
    scale_factor = 8  # Für A2
    scale = 1         # Standardabweichung

    # Statistikdaten vorbereiten
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

    # Nur die gewünschte Partei auswählen
    df_stats = df_stats[df_stats["Partei"] == partei_name]

    # Leeres Diagramm starten
    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        farbe = farben.get(partei_name, "gray")

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei_name,
            hovertemplate=(
                f"<b>{partei_name}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"
            ),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei_name],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    # Layout und Achsenbeschriftung
    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=f"{partei_name} – Position mit Ellipse ({scale}σ)",
            x=0.5,
            xanchor="center",
            font=dict(size=20 * scale_factor)
        ),
        font=dict(size=8 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=400, b=100, l=100, r=100),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )

    # Datei speichern
    filename = f"{save_path_base}\\ellipsen_a2_{partei_name.lower().replace(' ', '_')}.pdf"
    fig.write_image(filename, width=7016, height=4960)
    print(f"{partei_name}: gespeichert unter {filename}")


In [103]:
# Funktion anwenden
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck"

for partei in parteien:
    plot_partei_ellipse(partei, analyse, farben, save_path)

SPD: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\ellipsen_a2_spd.pdf
CDU: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\ellipsen_a2_cdu.pdf
FDP: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\ellipsen_a2_fdp.pdf
GRÜNE: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\ellipsen_a2_grüne.pdf
DIE LINKE: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\ellipsen_a2_die_linke.pdf
AfD: gespeichert unter C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\pdf-a2-querformat-ausdruck\el

## 2. PNG. Webseite und Social Media, 1600 x 1100 (anpassen über width und height)

In [104]:
def plot_partei_ellipse_png(parteien_liste, analyse, farben, save_path_base, all_together=False):
    scale_factor = 2  # Für Web reicht Faktor 2-3 meist aus (bei Bedarf anpassen)
    scale = 1

    # Statistikdaten vorbereiten
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()

    # Parteien filtern
    if not all_together:
        df_stats = df_stats[df_stats["Partei"].isin(parteien_liste)]

    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei,
            hovertemplate=(
                f"<b>{partei}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"
            ),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))

        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    # Titel für Einzelpartei oder alle zusammen
    if all_together:
        title_str = "Alle Parteien – Positionen mit Ellipsen (1σ)"
        filename = f"{save_path_base}\\ellipsen_webseite_alle_parteien.png"
    else:
        title_str = f"{parteien_liste[0]} – Position mit Ellipse (1σ)"
        filename = f"{save_path_base}\\ellipsen_webseite_{parteien_liste[0].lower().replace(' ', '_')}.png"

    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=title_str,
            x=0.5,
            xanchor="center",
            font=dict(size=20 * scale_factor)
        ),
        font=dict(size=10 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=120, b=60, l=60, r=60),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )

    # Größe für Web (z.B. 1600x1100px)
    fig.write_image(filename, width=1600, height=1100)
    print(f"PNG gespeichert: {filename}")

In [105]:
# Anwendung für einzelne Parteien:
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path_png = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100"

for partei in parteien:
    plot_partei_ellipse_png([partei], analyse, farben, save_path_png, all_together=False)

# Anwendung für alle zusammen:
plot_partei_ellipse_png(parteien, analyse, farben, save_path_png, all_together=True)

PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100\ellipsen_webseite_spd.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100\ellipsen_webseite_cdu.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100\ellipsen_webseite_fdp.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100\ellipsen_webseite_grüne.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\png-social media und webseite-1600x1100\ellipsen_webseite_die_linke.png
PNG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\vis

## 3. SVG (vektorbasiert, skalierbar), 800x600

ggf. anpassen:
für WEB: 1600x1100 (für header oä)
für DRUCK: z. B. 3508×2480 für A4, 7016×4960 für A2 bei 300 dpi

Vorsicht: ggf. scale_factor bearbeiten

In [106]:
def plot_partei_ellipse_svg(parteien_liste, analyse, farben, save_path_base, all_together=False,
                            width=1600, height=1100, scale_factor=2,
                            title_str_override=None, filename_override=None):
    scale = 1
    x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"})
    y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"})
    df_stats = pd.concat([x_stats, y_stats], axis=1).dropna().reset_index()
    if not all_together:
        df_stats = df_stats[df_stats["Partei"].isin(parteien_liste)]

    fig = go.Figure()
    angles = np.linspace(0, 2 * np.pi, 100)

    for _, row in df_stats.iterrows():
        x0, y0 = row["X"], row["Y"]
        a, b = row["X_std"], row["Y_std"]
        partei = row["Partei"]
        farbe = farben.get(partei, "gray")
        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0],
            mode='markers',
            marker=dict(size=15 * scale_factor, color=farbe),
            name=partei,
            hovertemplate=(f"<b>{partei}</b><br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<extra></extra>"),
            showlegend=False
        ))
        fig.add_trace(go.Scatter(
            x=[x0],
            y=[y0 + 0.08],
            mode="text",
            text=[partei],
            textfont=dict(size=14 * scale_factor, color="black"),
            showlegend=False,
            hoverinfo="skip"
        ))
        if a > 0 and b > 0:
            xs = x0 + scale * a * np.cos(angles)
            ys = y0 + scale * b * np.sin(angles)
            fig.add_trace(go.Scatter(
                x=xs, y=ys,
                fill="toself",
                mode="lines",
                line=dict(color=farbe, width=1),
                fillcolor=farbe,
                opacity=0.2,
                hoverinfo="skip",
                showlegend=False
            ))

    if all_together:
        title_str = title_str_override or "Alle Parteien – Positionen mit Ellipsen (1σ)"
        filename = filename_override or f"{save_path_base}\\ellipsen_webseite_alle_parteien.svg"
    else:
        title_str = title_str_override or f"{parteien_liste[0]} – Position mit Ellipse (1σ)"
        filename = filename_override or f"{save_path_base}\\ellipsen_webseite_{parteien_liste[0].lower().replace(' ', '_')}.svg"

    fig.update_layout(
        hovermode="closest",
        title=dict(
            text=title_str,
            x=0.5,
            xanchor="center",
            font=dict(size=18 * scale_factor)
        ),
        font=dict(size=8 * scale_factor),
        xaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        yaxis=dict(
            title="",
            range=[-1.2, 1.2],
            zeroline=True,
            showgrid=True,
            gridcolor="lightgrey",
            showline=True,
            tickmode='array',
            tickvals=[-1, 1],
            ticktext=["−1", "1"]
        ),
        plot_bgcolor="#e5ecf6",
        margin=dict(t=120, b=60, l=60, r=60),
        annotations=[
            dict(text="SOZIAL PROGRESSIV", x=0, y=1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="SOZIAL KONSERVATIV", x=0, y=-1.1, showarrow=False, xref="x", yref="y", font=dict(size=16 * scale_factor)),
            dict(text="LINKS", x=-1.1, y=0, showarrow=False, xref="x", yref="y", textangle=-90, font=dict(size=16 * scale_factor)),
            dict(text="RECHTS", x=1.1, y=0, showarrow=False, xref="x", yref="y", textangle=90, font=dict(size=16 * scale_factor)),
        ]
    )
    fig.write_image(filename, width=width, height=height)
    print(f"SVG gespeichert: {filename}")


In [107]:
parteien = ["SPD", "CDU", "FDP", "GRÜNE", "DIE LINKE", "AfD", "BSW"]
save_path_svg = r"C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600"

# Einzelne Parteien (Standardgröße für Web, beliebig anpassen)
for partei in parteien:
    plot_partei_ellipse_svg([partei], analyse, farben, save_path_svg, width=800, height=600, scale_factor=1)

# Alle zusammen
plot_partei_ellipse_svg(parteien, analyse, farben, save_path_svg, all_together=True, width=800, height=600, scale_factor=1)


SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_spd.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_cdu.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_fdp.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_grüne.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_die_linke.svg
SVG gespeichert: C:\Users\paula\OneDrive\Documents\10_Arbeit\2025\1-wahlomat-fes-sachsen\visuals-wahlomat\1-sigma\svg-vektor-800x600\ellipsen_webseite_afd.svg
SVG gespeichert: C:\Users\paula\OneDri

## 4. ggf anpassen für bestimmte Social Media Formate:

📱 Formate für Social Media
1. Twitter/X & Facebook:
Empfohlen: 1200 × 675 px (Verhältnis ca. 16:9)

Alternativ: 1600 × 900 px (etwas mehr Details)

2. Instagram Feed:
Quadratisch: 1080 × 1080 px

Hochformat: 1080 × 1350 px (besser für Sichtbarkeit im Feed)

Querformat: 1080 × 566 px (weniger Fläche im Feed sichtbar)

3. LinkedIn:
Empfohlen: 1200 × 627 px

# std kategorisieren

## funktion zur kategorisieurng

In [108]:
x_stats = analyse[analyse["Achse"] == "X"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "X", "std": "X_std"}).reset_index()
y_stats = analyse[analyse["Achse"] == "Y"].groupby("Partei")["Wert"].agg(["mean", "std"]).rename(columns={"mean": "Y", "std": "Y_std"}).reset_index()

In [109]:
print(y_stats)
print("")
print(x_stats)

      Partei         Y     Y_std
0        AfD -0.652174  0.714060
1        BSW  0.043478  1.021508
2        CDU -0.391304  0.891328
3  DIE LINKE  0.826087  0.576208
4        FDP  0.043478  0.975997
5      GRÜNE  0.695652  0.634950
6        SPD  0.304348  0.764840

      Partei         X     X_std
0        AfD  0.066667  0.961150
1        BSW -0.200000  1.014185
2        CDU  0.733333  0.593617
3  DIE LINKE -0.733333  0.703732
4        FDP  0.733333  0.593617
5      GRÜNE -0.466667  0.743223
6        SPD -0.400000  0.910259


In [110]:
# Funktion zur Kategorisierung der Standardabweichung
def kategorisiere_std(std_wert, min_std, max_std):  # Parameter: std_wert, min_std, max_std
    intervall = max_std - min_std                   # Spannweite der Standardabweichung
    if std_wert < min_std + 0.33 * intervall:
        return "homogen"
    elif std_wert < min_std + 0.66 * intervall:
        return "mittel"
    else:
        return "heterogen"

In [111]:
# Y_std klassifizieren
min_y, max_y = y_stats['Y_std'].min(), y_stats['Y_std'].max()
y_stats['Y_Kat'] = y_stats['Y_std'].apply(lambda x: kategorisiere_std(x, min_y, max_y))

# X_std klassifizieren
min_x, max_x = x_stats['X_std'].min(), x_stats['X_std'].max()
x_stats['X_Kat'] = x_stats['X_std'].apply(lambda x: kategorisiere_std(x, min_x, max_x))

# Zusammenführen
stats_merged = pd.merge(
    x_stats[['Partei', 'X_std', 'X_Kat']],
    y_stats[['Partei', 'Y_std', 'Y_Kat']],
    on='Partei'
)

# Ausgabe
print(stats_merged)


      Partei     X_std      X_Kat     Y_std      Y_Kat
0        AfD  0.961150  heterogen  0.714060    homogen
1        BSW  1.014185  heterogen  1.021508  heterogen
2        CDU  0.593617    homogen  0.891328  heterogen
3  DIE LINKE  0.703732    homogen  0.576208    homogen
4        FDP  0.593617    homogen  0.975997  heterogen
5      GRÜNE  0.743223     mittel  0.634950    homogen
6        SPD  0.910259  heterogen  0.764840     mittel


## visualisierung der kategorisierung

In [112]:
# Mapping in Emojis
kat_to_emoji = {
    'homogen': '🟢',
    'mittel': '🟡',
    'heterogen': '🔴'
}

# Emojis ergänzen
stats_merged['X_Emoji'] = stats_merged['X_Kat'].map(kat_to_emoji)
stats_merged['Y_Emoji'] = stats_merged['Y_Kat'].map(kat_to_emoji)

# Interaktive Tabelle
fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Partei", "σ X-Achse", "σ Y-Achse"],
        fill_color='#ced4c8',
        align='left',
        font=dict(color='black', size=14, family='Arial'),
    ),
    cells=dict(
        values=[
            stats_merged['Partei'],
            stats_merged['X_Emoji'],
            stats_merged['Y_Emoji']
        ],
        fill_color='white',
        align='left',
        height=30,
        font=dict(color='black', size=12, family='Arial')
    )
)])

fig.update_layout(
    title=dict(
        text="🧭 Streuung der Parteipositionen",
        x=0.5,  # 👈 Mitte (0 = links, 1 = rechts)
        xanchor='center',  # Verankert den Text an der Mitte
        font=dict(size=18)  # Optional: Schriftgröße des Titels
    ),
    width=600,
    height=400
)

fig.show()